In [40]:
from queue import Queue
import numpy as np

# Reading communities and edge data

In [41]:
community_membership = np.loadtxt('community_membership_2K.csv', delimiter=',', dtype=np.int32)
edges = np.loadtxt('edges_sampled_2K.csv', delimiter=',', dtype=np.int32)

In [42]:
edges

array([[27228, 30121],
       [27228, 29697],
       [27228, 28142],
       ...,
       [26661, 30121],
       [26661, 27193],
       [26661, 33400]], dtype=int32)

In [43]:
unique_edges = np.unique(edges)

num_vertices = unique_edges.shape[0]

edge_to_id = dict(zip(unique_edges, range(num_vertices + 1)))

edges = np.vectorize(edge_to_id.get)(edges)

In [44]:
edges

array([[111, 134],
       [111, 125],
       [111, 116],
       ...,
       [ 92, 134],
       [ 92, 109],
       [ 92, 159]])

In [45]:
def dfs(run, vertex, visited, neighbors):
        # Mark the current vertex as visited
        visited[vertex] = True

        # Store the vertex to list
        run.append(vertex)

        # Repeat for all vertices adjacent
        # to this vertex v
        for i in neighbors[vertex]:
            if i is None: continue
            if not visited[i]:
                # Update the list
                run = dfs(run, i, visited, neighbors)
        return run

def get_communities(edges):
    unique_edges = np.unique(edges)
    num_vertices = unique_edges.shape[0]
    neighbors = np.empty((num_vertices,), dtype=object)
    for i, v in enumerate(neighbors): neighbors[i] = [v]
    for edge in edges:
        n1 = edge[0]
        n2 = edge[1]
        neighbors[n1].append(n2)
        neighbors[n2].append(n1)

    visited = []
    connected_comps = []
    for i in range(num_vertices):
        visited.append(False)
    for v in range(num_vertices):
        if not visited[v]:
            temp = []
            connected_comps.append(dfs(temp, v, visited, neighbors))
    return connected_comps

In [46]:
def edge_betweenness(edges: np.ndarray) -> dict:

    unique_edges = np.unique(edges)

    num_vertices = unique_edges.shape[0]

    neighbors = np.empty((num_vertices,), dtype=object)
    for i, v in enumerate(neighbors):
        neighbors[i] = [v]
    for edge in edges:
        n1 = edge[0]
        n2 = edge[1]
        neighbors[n1].append(n2)
        neighbors[n2].append(n1)
    tran = edges.T
    new_edges = list(zip(tran[0], tran[1]))
    edge_mapping = dict.fromkeys(range(num_vertices), 0.0)
    edge_mapping.update(dict.fromkeys(new_edges, 0.0))
    for s in range(num_vertices):
        stack = []
        P = np.empty((num_vertices,), dtype=object)
        for i, v in enumerate(P): P[i] = [v]
        sigma = np.zeros(num_vertices)
        sigma[s] = 1
        d = np.full(num_vertices, -1)
        d[s] = 0
        queue = Queue()
        queue.put(s)
        while not queue.empty():
            v = queue.get()
            stack.append(v)
            for w in neighbors[v]:
                if w is None: 
                    continue
                if d[w] < 0:
                    queue.put(w)
                    d[w] = d[v] + 1
                if d[w] == d[v] + 1:
                    sigma[w] += sigma[v]
                    P[w].append(v)
        delta = np.zeros(num_vertices)
        while len(stack) > 0:
            w = stack.pop()
            for v in P[w]:
                if v is None: continue
                update = sigma[v] / sigma[w] * (1 + delta[w])
                if (v, w) not in edge_mapping:
                    edge_mapping[(w, v)] += update
                else:
                    edge_mapping[(v, w)] += update
                delta[v] += update
                if w != s: edge_mapping[w] += delta[w]
    for vertex in range(num_vertices): 
        del edge_mapping[vertex]  
    for v in edge_mapping: 
        edge_mapping[v] *= 0.5  
    
    return edge_mapping

In [47]:
print(f'{0} edges removed, {len(get_communities(edges))} communities')

num_communities = len(get_communities(edges))
new_communities = num_communities
num_removed = 0
for i in range(3):
    while new_communities==num_communities:
        betweenness = edge_betweenness(edges)
        max_edge = max(betweenness, key=betweenness.get)
        # Remove the max edge
        edges = np.delete(edges, np.where((edges[:,0]==max_edge[0]) & (edges[:,1]==max_edge[1]))[0],axis=0)
        new_communities = len(get_communities(edges))
        num_removed += 1
        print(f'Removing another edge, total {num_removed}')
    print(f'{num_removed} edges removed, {new_communities} communities')
    num_communities = new_communities

0 edges removed, 5 communities
Removing another edge, total 1
Removing another edge, total 2
2 edges removed, 6 communities
Removing another edge, total 3
3 edges removed, 7 communities
Removing another edge, total 4
4 edges removed, 8 communities
